In [1]:
# Install packages

!pip install lightning-bolts

     |████████████████████████████████| 282 kB 7.7 MB/s 
     |████████████████████████████████| 1.0 MB 43.5 MB/s 
     |████████████████████████████████| 329 kB 56.5 MB/s 
     |████████████████████████████████| 596 kB 57.8 MB/s 
     |████████████████████████████████| 132 kB 53.1 MB/s 
     |████████████████████████████████| 829 kB 44.7 MB/s 
     |████████████████████████████████| 1.1 MB 69.1 MB/s 
     |████████████████████████████████| 160 kB 72.2 MB/s 
     |████████████████████████████████| 192 kB 60.2 MB/s 
     |████████████████████████████████| 271 kB 51.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=aeb3b1e7e8a472fedfce41d3141bfb4e1c3bf0c3fa129c3e1cfd93d90ba75308
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled

In [6]:
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import os
from torchvision import datasets, transforms

# transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# data
dataset_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
dataloader_train = DataLoader(dataset_train, batch_size=64)

In [16]:
from pl_bolts.models.autoencoders.components import ResNetEncoder, EncoderBlock

class CustomEncoder(ResNetEncoder):
    def __init__(self, block, layers):
        super().__init__(block, layers)
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)

In [17]:
from pl_bolts.models.autoencoders import AE
import torch.nn as nn

class CustomAE(AE):
    def init_encoder(self, hidden_dim, latent_dim, input_width, input_height):
        encoder = CustomEncoder(EncoderBlock, [2, 2, 2, 2])
        return encoder

model = CustomAE(28)

In [18]:
import torch
from pytorch_lightning import Trainer

trainer = Trainer(gpus=int(torch.cuda.is_available()))
trainer.fit(model, dataloader_train)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:118: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name    | Type          | Params
------------------------------------------
0 | encoder | ResNetEncoder | 11.2 M
1 | decoder | ResNetDecoder | 8.6 M 
2 | fc      | Linear        | 131 K 
------------------------------------------
19.9 M    Trainable params
0         Non-trainable params
19.9 M    Total params
79.703    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

RuntimeError: ignored